In [61]:
import time
import hashlib
import pandas as pd

In [77]:
'''
Setting up the input data for the blocks 
Each row in this dataframe corresponds to a new block in the chain
The dataframe will also store the metadata about block once we are done mining each bloc
'''
input_data = pd.DataFrame(columns={'data','data_id','prev_block_id','curr_block_id', 'mining_time','nonce'})
input_data['data'] = ["Let's create the first block in the chain", "This is the second block", "This is the third block", "This is the fourth block"]
input_data

,prev_block_id,nonce,curr_block_id,data,mining_time,data_id
0,NaN,NaN,NaN,Let's create the first block in the chain,NaN,NaN
1,NaN,NaN,NaN,This is the second block,NaN,NaN
2,NaN,NaN,NaN,This is the third block,NaN,NaN
3,NaN,NaN,NaN,This is the fourth block,NaN,NaN


In [102]:
#How many leading zeros count (lzc) should be there in the acceptable hash for current block ID
'''User can change this, please note that the larger this number the longer it will take for the program to finish running'''
lzc = 6

In [103]:
'''
This function is called for each row (therefore each block)
::input::
data_to_hash: the data to be stored in the block
prev_block_id: the previous block's id, for the first block it will be empty
::output::
data_id: the Data ID for the datain the block
prev_block_id: the previous block ID
current_block_id: the current block ID
nonce: the nonce value, which also represent the number of attempts it took to mine the block
mining_time: the time in seconds that it took to mine the block
'''
def create_block(data_to_hash, prev_block_id=None):
    result = hashlib.sha256(data_to_hash.encode())
    data_id = result.hexdigest()
    nonce = 0
    start_time = time.time()
    hash_output = data_id
    if hash_output[:lzc]!='0'*lzc:
        while hash_output[:lzc]!='0'*lzc:
            if prev_block_id:
                nonced_data = prev_block_id+'\n'+data_id+'\n'+str(nonce)
            else:
                nonced_data = data_id+'\n'+str(nonce)
            result = hashlib.sha256(nonced_data.encode())
            hash_output = result.hexdigest()
            nonce = nonce+1
    current_block_id = hash_output
    mining_time = time.time() - start_time
    nonce
    return data_id, prev_block_id, current_block_id, mining_time, nonce

In [104]:
'''Calling the block mining function for each row of the input'''
for index, row in input_data.iterrows():
    data_to_hash = row['data']
    if index!=0:
        prev_block_id = input_data.at[index-1,'curr_block_id']
        mining_output = create_block(data_to_hash,prev_block_id)
    else:
        mining_output = create_block(data_to_hash)
    row['data_id'] = mining_output[0]
    row['prev_block_id'] = mining_output[1]
    row['curr_block_id'] = mining_output[2]
    row['mining_time'] = mining_output[3]
    row['nonce'] = mining_output[4]

In [105]:
'''Display the output once all blocks are mined'''
input_data[['mining_time','nonce','data','data_id','prev_block_id','curr_block_id']]

,mining_time,nonce,data,data_id,prev_block_id,curr_block_id
0,21.0116,13688670,Let's create the first block in the chain,50c6c115afd9e169b3528593fc6411161ed4b1c88a48e5...,None,00000040aea398db137c35ad7f80ffef6d2c4160c8622b...
1,29.6818,17329931,This is the second block,37764951221c870d565e13ac86a387a2823e2923a3aceb...,00000040aea398db137c35ad7f80ffef6d2c4160c8622b...,000000aa1bf704ea64129c232da94c302cebdef690ca88...
2,18.2316,10665238,This is the third block,029fe325fbde424eeef2013f1d335f5923e543728972a5...,000000aa1bf704ea64129c232da94c302cebdef690ca88...,00000042c367dee8504e18b7b60fe9b31194b7d8c61750...
3,0.825187,483151,This is the fourth block,1b5f8f54cd8bf79af29c8195b4f9561f242ec7d022d162...,00000042c367dee8504e18b7b60fe9b31194b7d8c61750...,000000b32c187fb2728fccfcdbf0017dd10053bf7d0a7c...
